In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


In [2]:
excel_file = r"vi_prod_class_22-10.xlsx"  # Đường dẫn đến file Excel của bạn

In [3]:
df_1 = pd.read_excel(excel_file, sheet_name = r'tranformed Kqxd trước mã số', skiprows=1).iloc[:,:3].fillna('')
df_2 = pd.read_excel(excel_file, sheet_name = r'transformed Phân loại hàng hóa ', skiprows=1).iloc[:,:3].fillna('')

df = pd.concat([df_1,df_2])
df.columns = ['link','doc_no','doc_date']
df = df.drop_duplicates()
df['doc_name'] = df['doc_no'].replace('/', '_', regex=True).replace('-', '_', regex=True)+'_'+df['doc_date'].replace('-', '_', regex=True)


In [4]:
df.head()

,link,doc_no,doc_date,doc_name
0,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3664/TB-TCHQ,29-07-2024,3664_TB_TCHQ_29_07_2024
1,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3349/TB-TCHQ,10-07-2024,3349_TB_TCHQ_10_07_2024
2,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3313/TB-TCHQ,08-07-2024,3313_TB_TCHQ_08_07_2024
3,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3312/TB-TCHQ,08-07-2024,3312_TB_TCHQ_08_07_2024
4,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,2710/TB-TCHQ,12-06-2024,2710_TB_TCHQ_12_06_2024


In [5]:
# df.to_excel("link_for_download_pdf.xlsx",index = True)

In [6]:
# Đường dẫn để đăng nhập
login_url = 'https://thuvienphapluat.vn/page/ajaxcontroler.aspx'

# Dữ liệu cần gửi để đăng nhập
payload = {
    'l_txtUser': 'UNICUSVN',
    'l_txtPass': 'UNIUNI',
    'action': 'Login'
}

# Headers cần thiết cho yêu cầu
headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/x-www-form-urlencoded',
    'DNT': '1',
    'Origin': 'https://thuvienphapluat.vn',
    'Pragma': 'no-cache',
    'Referer': 'https://thuvienphapluat.vn/page/login.aspx?',
    'Sec-CH-UA': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    'Sec-CH-UA-Mobile': '?0',
    'Sec-CH-UA-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

# Khởi tạo session để giữ cookie
session = requests.Session()

# Gửi yêu cầu POST để đăng nhập
login_response = session.post(login_url, data=payload, headers=headers)
# Kiểm tra đăng nhập thành công
if login_response.ok:
    print("Đăng nhập thành công!")
else:
    print("Đăng nhập thất bại:", login_response.status_code)


Đăng nhập thành công!


In [7]:
def get_downloadlink(link):
    try:
        # Bước 1: Gửi yêu cầu tới link
        response = session.get(link)
        if response.status_code != 200:
            raise Exception(f"Lỗi: trạng thái yêu cầu không thành công ({response.status_code})")
    except requests.RequestException as e:
        print(f"Lỗi khi gửi yêu cầu tới {link}: {e}")
        raise  # Dừng hàm nếu có lỗi trong yêu cầu

    try:
        # Bước 2: Phân tích HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        find_pdf_link = soup.find('div', id='tab8').find('a', id='ctl00_Content_ThongTinVB_pdfHyperLink')
        
        
        if not find_pdf_link:
            raise Exception("Không có link PDF trong div 'tab8'.")
    except Exception as e:
        print(f"Lỗi khi tìm link PDF: {e}")
        raise  # Dừng hàm nếu có lỗi trong quá trình tìm link PDF

    try:
        # Bước 3: Tìm tất cả các thẻ script và khớp mẫu URL
        all_script = soup.find_all('script')
        pattern = r'/documents/download\.aspx\?id=[\w%]+'

        if not all_script:
            raise Exception("Không tìm thấy thẻ script nào trong trang.")

        for text in all_script:
            if '__urldl' in text.get_text():
                matches = re.findall(pattern, text.get_text())
                if matches:
                    download_url = f'https://thuvienphapluat.vn{matches[-1]}&part=-0'
                    print("Download URL:", download_url)
                    return download_url
        raise Exception("Không tìm thấy link PDF trong các thẻ script.")
    except Exception as e:
        print(f"Lỗi khi tìm các thẻ script và khớp mẫu URL: {e}")
        raise  # Dừng hàm nếu có lỗi trong quá trình xử lý các thẻ script

In [8]:
def download_pdf(download_link,file_name,idx):
    # path = r"D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_pdf\\"
    path = r"F:\Onedrive Uni\OneDrive - UNI CONSULTING\Eximhow_pdf\test_1\\"
    # Gửi yêu cầu tải file PDF
    response = session.get(download_link, headers=headers, stream=True)

    # Kiểm tra nếu yêu cầu thành công và lưu file
    if response.ok:
        
        with open(f"{path}{file_name}_{idx}.pdf", "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Tải file thành công và lưu thành {file_name}.pdf")
    else:
        print("Lỗi khi tải file:", response.status_code)


In [9]:
len(df)

6595

In [10]:
df.iloc[20:21,:]

,link,doc_no,doc_date,doc_name
20,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3874/TB-TCHQ,25-07-2023,3874_TB_TCHQ_25_07_2023


In [19]:
for idx,row in df.iterrows():
    print (idx)
    print(row[0])
    download_link= get_downloadlink(row[0])
    print(row[3])
    file_name = row[3]
    if download_link : 
        try:
            download_pdf(download_link,file_name,idx)
        except Exception as e:
            print(f"Lỗi khi tìm tất cả các thẻ script: {e}")


20
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3874-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-doi-voi-Fish-Pie-Mix-574649.aspx


C:\Users\cuongnm\AppData\Local\Temp\ipykernel_10080\1281177218.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[0])
C:\Users\cuongnm\AppData\Local\Temp\ipykernel_10080\1281177218.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  download_link= get_downloadlink(row[0])


Matches found: ['/documents/download.aspx?id=NueMOry9VEObnWTAnNoOOg%3d%3d']
Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=NueMOry9VEObnWTAnNoOOg%3d%3d&part=-0
3874_TB_TCHQ_25_07_2023


C:\Users\cuongnm\AppData\Local\Temp\ipykernel_10080\1281177218.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[3])
C:\Users\cuongnm\AppData\Local\Temp\ipykernel_10080\1281177218.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  file_name = row[3]


Tải file thành công và lưu thành 3874_TB_TCHQ_25_07_2023.pdf
21
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3873-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-doi-voi-Fish-Pie-Mix-574648.aspx
Matches found: ['/documents/download.aspx?id=h5XWvEQDUpofXwiPMSYBHA%3d%3d']
Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=h5XWvEQDUpofXwiPMSYBHA%3d%3d&part=-0
3873_TB_TCHQ_25_07_2023
Tải file thành công và lưu thành 3873_TB_TCHQ_25_07_2023.pdf
22
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3831-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-Coating-Guide-574647.aspx
Matches found: ['/documents/download.aspx?id=kOBNSx2z4ARW71t%2fTHCBnw%3d%3d']
Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=kOBNSx2z4ARW71t%2fTHCBnw%3d%3d&part=-0
3831_TB_TCHQ_21_07_2023
Tải file thành công và lưu thành 3831_TB_TCHQ_21_07_2023.pdf
23
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-2905-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-Soy-dietary-fi

MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?

In [12]:
import concurrent.futures
import queue

# Tạo hàng đợi để đảm bảo thứ tự xử lý các hàng trong DataFrame
task_queue = queue.Queue()

# Đưa tất cả các dòng vào hàng đợi
for row in df.iloc[20:,:].iterrows():
    task_queue.put(row)

# Hàm để xử lý từng hàng trong DataFrame
def download_worker():
    while not task_queue.empty():
        idx, row_data = task_queue.get()
        print(idx)
        print(row_data[0])
        download_link = get_downloadlink(row_data[0])
        print(row_data[3])
        file_name = row_data[3]
        if download_link:
            try:
                download_pdf(download_link, file_name, idx)
            except Exception as e:
                print(f"Lỗi khi tìm tất cả các thẻ script: {e}")
        task_queue.task_done()

# Sử dụng ThreadPoolExecutor để tạo multithreading



In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    # Tạo 50 threads để thực thi `download_worker`
    futures = [executor.submit(download_worker) for _ in range(50)]

    # Chờ đến khi tất cả các task trong queue được xử lý
    task_queue.join()

20
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3874-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-doi-voi-Fish-Pie-Mix-574649.aspx
21
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3873-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-doi-voi-Fish-Pie-Mix-574648.aspx
22
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3831-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-Coating-Guide-574647.aspx
23
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-2905-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-Soy-dietary-fiber-570261.aspx
24
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-2765-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-Binh-tich-ap-570256.aspx
25
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-2145-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-doi-voi-SOLSPERSE-88000-567736.aspx
26
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-1771-TB-TCHQ-2023-ket-qua-xac-dinh-truoc-ma-so-Thuoc-Rodogyl-564614.aspx
27
https://thuvienpha

C:\Users\cuongnm\AppData\Local\Temp\ipykernel_16412\113599657.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row_data[0])
C:\Users\cuongnm\AppData\Local\Temp\ipykernel_16412\113599657.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  download_link = get_downloadlink(row_data[0])


Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=kOBNSx2z4ARW71t%2fTHCBnw%3d%3d&part=-0
3831_TB_TCHQ_21_07_2023


C:\Users\cuongnm\AppData\Local\Temp\ipykernel_16412\113599657.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row_data[3])
C:\Users\cuongnm\AppData\Local\Temp\ipykernel_16412\113599657.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  file_name = row_data[3]


Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=KE3LsqqD%2bqfS144ZwvWZPQ%3d%3d&part=-0
4054_TB_TCHQ_23_08_2024
Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=ZCexq4fJ3QjZ79cS6pEZsA%3d%3d&part=-0
4100_TB_TCHQ_26_08_2024
Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=f5UAqNKEuZrYLg3SnGFXig%3d%3d&part=-0
4051_TB_TCHQ_23_08_2024
Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=1ChQ1JtTfoyRWet1nRDHRA%3d%3d&part=-0
4098_TB_TCHQ_26_08_2024
Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=EJzRf72a7F1k91PhklpIvg%3d%3d&part=-0
4097_TB_TCHQ_26_08_2024
Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=s3uWldd%2flI%2bVHVexmmgamQ%3d%3d&part=-0
3055_TB_TCHQ_26_07_2022
Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=Vf5BEvOU0Q2pnN9uRLGxJA%3d%3d&part=-0
1663_TB_TCHQ_11_04_2023
Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=WCO1ck3%2btl7%2bMw%2fF6

In [11]:
test = get_downloadlink('https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3347-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-doi-voi-Hat-nhua-PET-tai-sinh-617395.aspx')

Download URL: https://thuvienphapluat.vn/documents/download.aspx?id=w7JqBPncodrBJLLX6Dc%2fVQ%3d%3d&part=-0


In [69]:
response = session.get('https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3347-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-doi-voi-Hat-nhua-PET-tai-sinh-617395.aspx')

In [70]:
soup = BeautifulSoup(response.text, 'html.parser')

In [71]:
soup


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head id="ctl00_Head1"><link href="../../favicon.ico" rel="shortcut icon"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="https://cdn.thuvienphapluat.vn/uploads/Images/thumbnail_TVPL.png" property="og:image"/><meta content="345074462254483" property="fb:admins"/><meta content="388492657962135" property="fb:app_id"/><meta content="345074462254483" property="fb:page_id"/><meta content="no-store" http-equiv="Cache-Control"/><title>
	Thông báo 3347/TB-TCHQ năm 2024 về kết quả xác định trước mã số đối với Hạt nhựa PET tái sinh do Tổng cục Hải quan ban hành
</title><meta content="THƯ VIỆN PHÁP LUẬT" property="og:site_name"/><meta content="Thông báo 3347/TB-TCHQ năm 2024 về kết quả xác định trước mã số đối với Hạt nhựa PET tái sinh do Tổng cục Hải quan ban hành" property="og:title"/><meta content="websit

In [72]:
find_pdf_link = soup.find('div', id='tab8').find('a', id='ctl00_Content_ThongTinVB_pdfHyperLink')

In [64]:
download_pdf(test,"test",10)

Tải file thành công và lưu thành test.pdf
